# JAX中的伪随机数

*作者： Matteo Hessel & Rosalia Schneider*

这一章中我们主要关注伪随机数生成(PRNG: pseudo random number generation)；换句话说，就是通过算法来生成数字序列的过程，这个序列的性质近似于从恰当分布中采样的随机数字序列的性质。

PRNG生成的序列并不是真实的随机，因为它们实际上由初始值决定，也就是我们通常提到的种子 `seed`，随机采样的每个步骤都是某个状态 `state` 的确定性函数，这个状态会从一个样本转移到下一个样本。

在任何机器学习和科学计算框架中，伪随机数生成都是很重要的一个组成部分。一般来说，JAX尽可能与NumPy兼容，但伪随机数生成是一个显著的例外。

为了更好理解随机数生成中JAX和NumPy的区别，在这一章中两种方法都会有所涉及。

## NumPy中的随机数

在NumPy中，伪随机中生成由 `numpy.random` 模块原生支持。伪随机数生成基于全局状态 `state`，它可以由 `random.seed(SEED)` 来指定初始状态。

In [1]:
import numpy as np

np.random.seed(0)

我们可以使用一下命令来检查状态的内容：

In [2]:
def print_truncated_random_state():
    """
    To avoid spamming the outputs, print only part of the state.
    为了避免输出过多冗杂输出，只打印部分状态
    """
    
    full_random_state = np.random.get_state()
    print(str(full_random_state)[:460], '...')
    
print_truncated_random_state()

('MT19937', array([         0,          1, 1812433255, 1900727105, 1208447044,
       2481403966, 4042607538,  337614300, 3232553940, 1018809052,
       3202401494, 1775180719, 3192392114,  594215549,  184016991,
        829906058,  610491522, 3879932251, 3139825610,  297902587,
       4075895579, 2943625357, 3530655617, 1423771745, 2135928312,
       2891506774, 1066338622,  135451537,  933040465, 2759011858,
       2273819758, 3545703099, 2516396728, 127 ...


每次调用一个随机函数，`state`就更新一次：

In [3]:
np.random.seed(0)
print_truncated_random_state()
_ = np.random.uniform()
print_truncated_random_state()

('MT19937', array([         0,          1, 1812433255, 1900727105, 1208447044,
       2481403966, 4042607538,  337614300, 3232553940, 1018809052,
       3202401494, 1775180719, 3192392114,  594215549,  184016991,
        829906058,  610491522, 3879932251, 3139825610,  297902587,
       4075895579, 2943625357, 3530655617, 1423771745, 2135928312,
       2891506774, 1066338622,  135451537,  933040465, 2759011858,
       2273819758, 3545703099, 2516396728, 127 ...
('MT19937', array([2443250962, 1093594115, 1878467924, 2709361018, 1101979660,
       3904844661,  676747479, 2085143622, 1056793272, 3812477442,
       2168787041,  275552121, 2696932952, 3432054210, 1657102335,
       3518946594,  962584079, 1051271004, 3806145045, 1414436097,
       2032348584, 1661738718, 1116708477, 2562755208, 3176189976,
        696824676, 2399811678, 3992505346,  569184356, 2626558620,
        136797809, 4273176064,  296167901, 343 ...


NumPy 允许您在单个函数调用中对单个数字或整个数字向量进行采样。例如，您可以通过执行以下操作从均匀分布中采样一个向量中的三个标量：

In [4]:
np.random.seed(0)
print(np.random.uniform(size=3))

[0.5488135  0.71518937 0.60276338]


NumPy提供了一个 *顺序等效保证*，意思是单独对N个数字进行连续采样和对N个数字的向量采样将会产生相同的伪随机序列：

In [5]:
np.random.seed(0)
print("individually:", np.stack([np.random.uniform() for _ in range(3)]))

np.random.seed(0)
print("all at once:", np.random.uniform(size=3))

individually: [0.5488135  0.71518937 0.60276338]
all at once: [0.5488135  0.71518937 0.60276338]


## JAX中的随机数

JAX中的随机数生成和NumPy中的有很显著的不同。原因是NumPy中的PRNG设计很难同时保证JAX中的许多理想化属性，特别是代码应该具有：

1. 可复现化，
2. 可并行化，
3. 可向量化。

我们将在下面讨论原因。首先，我们先专注于基于全局状态的PRNG设计的含义。我们来看以下代码：

In [6]:
import numpy as np

np.random.seed(0)

def bar():
    return np.random.uniform()

def baz():
    return np.random.uniform()

def foo():
    return bar() + 2 * baz()

print(foo())

1.9791922366721637


函数 `foo` 将从一个均匀分布中的采样到的两个标量求和。如果像原生的Python那样，假设 `bar()`和 `baz()` 两个函数的执行顺序固定，则此时代码的输出只能满足第一个要求。这在NumPy中貌似不是个大问题，由于它已经被Python强制执行，但在JAX就行不通了。

要使以上代码在JAX中可重现，则将需要强制指定特性的执行顺序。这样的话将会违反第二个要求，因为在JAX中应该可以并行化 `bar` 和 `baz`，因为这两个函数并不相互依赖。所以为了避免这个问题，JAX不采用全局状态。随机函数显式采用一个状态，我们取而代之称之为 `key`。



In [7]:
from jax import random

key = random.PRNGKey(42)
print(key)

[ 0 42]


`key`只是一个 `(2,)` 形状的数组。

本质上，`Random key`只是 `Random seed` 的一个别称。但是，与其在NumPy中设置一次一样，在JAX中对随机函数的任何调用都需要指定一个 `key`。随机函数使用 `key`，但不对齐进行修改。向随机函数提供相同的 `key` 都将始终生成相同的样本：

In [8]:
print(random.normal(key))
print(random.normal(key))

-0.18471177
-0.18471177


**注意**: 通常我们不希望将相同的 `key` 提供给不同的随机函数，这样可能会导致相关联的结果。

**经验法则：永远不要重用 `key`** （除非您想要相同的输出）。

为了生成不同且独立的样本，每当您要调用随机函数时，都必须*自己手动* `split()` key：

In [9]:
print("old key", key)
new_key, subkey = random.split(key)
# The old key is discarded -- we must never use it again.
# 旧key被丢弃——我们不能重复使用它
del key 
normal_sample = random.normal(subkey)
print(r"    \---SPLIT--> new key    ", new_key)
print(r"            \--> new subkey ", subkey, "--> normal", normal_sample)
# The subkey is also discarded after use.
# subkey在使用后依然被丢弃
del subkey

# Note: you don't actually need to `del` keys -- that's just for emphasis.
# Not reusing the same values is enough.
# 注意：一般来说不需要真的 `del` key，这里只是为了强调
# 只要不使用相同值就行

# If we wanted to do this again, we would use new_key as the key.
# 如果我们想要再来一次，就直接使用 `new_key`
key = new_key  

old key [ 0 42]
    \---SPLIT--> new key     [2465931498 3679230171]
            \--> new subkey  [255383827 267815257] --> normal 1.3694694


`split()` 是一个确定性函数，他可以将一个key转换为几个独立的key（在伪随机的意义上）。我们将输出之一保留为 `new_key`，可以安全地使用唯一的额外key （称为 `subkey`）作为随机函数的输入，然后将其永久丢弃。

如果要从同一个正态分布中取得另一个样本，则可以再次拆分 `key`，以此类推。关键的是，您永远不会两次使用相同的 `PRNGKey`。由于 `split()` 将key作为其参数，因此在拆后时必须丢弃旧key。

我们将 `split(key)` 中输出的哪一部分称为 `key` 或者 `subkey` 并不重要，他们都是状态相同的伪随机数。我们使用 `key/subkey`的约定原因是为了跟踪他们在未来被使用的方式。 `subkey` 注定由随机函数直接消耗掉，而 `key` 可以保留在以后产生更多随机性。

通常，以上的例子可以更简洁地写成：


In [10]:
key, subkey = random.split(key)

这样将会自动消耗旧key。值得一提的是，不管你需要多少key，`split()` 函数都可以产生：

In [11]:
key, *forty_two_subkeys = random.split(key, num=43)

NumPy和JAX的随机模块之间的另一个区别与上述的 *顺序等效保证* 有关。与NumPy相同，JAX的随机模块也允许对数字向量进行采样。但是，JAX不提供*顺序等效保证*，因为这样做会干扰SIMD硬件上的向量化。

在下面的示例中，使用三个subkey从正太分布中采样三个值的结果和使用单个key指定 `shape=(3,)`的结果不同：

In [12]:
key = random.PRNGKey(42)
subkeys = random.split(key, 3)
sequence = np.stack([random.normal(subkey) for subkey in subkeys])
print("individually:", sequence)

key = random.PRNGKey(42)
print("all at once:", random.normal(key, shape=(3,)))

individually: [-0.04838832  0.10796154 -1.2226542 ]
all at once: [ 0.18693547 -1.2806505  -1.5593132 ]


注意，这里与上面我们的建议相反，在第二个示例中我们直接使用 `key` 作为`random.normal()` 的输入。这是因为我们不会在其他任何地方重复使用它，因此我们不会违反一次性使用原则。